<br><br>

## **Import necessary Python libraries and modules**

First, we will import necessary Python libraries and modules. These include as `gdown`, for downloading large files from Google Drive (where we will get our UCSD Goodreads reviews), as well as scikit-learn (`sklearn`) and PyTorch (`torch`), for various machine learning tools.

In [1]:

# For data manipulation and analysis
import pandas as pd
import numpy as np

# For machine learning tools and evaluation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report


# For deep learning
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
import torch



To use the HuggingFace [`transformers` Python library](https://huggingface.co/transformers/installation.html), we will install it with `pip`.

In [2]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 4.5 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 895 kB 51.3 MB/s 
     |████████████████████████████████| 6.8 MB 23.5 MB/s 
     |████████████████████████████████| 596 kB 36.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Once `transformers` is installed, we will import modules for `DistilBert`, a *distilled* or smaller version of a BERT model that runs more quickly and uses less computing power. This makes it ideal for those just getting started with BERT.

In [3]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

<br><br>

## **Set parameters and file paths**

In [5]:
# This is the name of the BERT model that we want to use. 
# We're using DistilBERT to save space (it's a distilled version of the full BERT model), 
# and we're going to use the cased (vs uncased) version.
model_name = 'distilbert-base-multilingual-cased'  

# This is the name of the program management system for NVIDIA GPUs. We're going to send our code here.
device_name = 'cuda'       

# This is the maximum number of tokens in any document sent to BERT.
max_length = 512                                                        

# This is the name of the directory where we'll save our model. You can name it whatever you want.
#cached_model_directory_name = 'ABSA_FineTuning_BERT'  
cached_model_directory_name = 'Polarity_2_noneut'  

In [6]:
#stiamo utlizzando la GPU?
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [7]:
train2=pd.read_csv("train2_noneut.csv")
train2

,Unnamed: 0,level_0,Sentence,Polarity_Classification
0,0,911,It's often choppy or unclear and I would imagi...,Negativa
1,1,40,But Nix doesn't downplay his brand of horrific...,Positiva
2,2,1068,He donated much of his wealth to the poor whic...,Positiva
3,3,621,"Highly recommended, especially for the younger...",Positiva
4,4,427,"The main character, Arthur is flawed and that ...",Negativa
...,...,...,...,...
67556,67556,28420,완전 재미는데,Positiva
67557,67557,90755,한쪽을 잃은 사람들은 현재와과거에서 상통하는걸 모르는 설탕가게(CJ)영화간부들,Positiva
67558,67558,9022,헐 너무 멋지다. 이영화는 혁명입니다.,Positiva
67559,67559,104179,오글.....,Negativa


In [8]:
test=pd.read_csv("test_noneut.csv")
test

,Unnamed: 0,level_0,Sentence,Polarity_Classification
0,0,22,"My darling,My hamburger is a good book if it d...",Positiva
1,1,1060,I highly recommend this book to any parent tha...,Positiva
2,2,567,Dr Tony Hill and DCI Jordan are back - hurrah!...,Positiva
3,3,741,I really liked this book because it was inter...,Positiva
4,4,765,"""Mister Monday"" reminds me more of ""The Ragwit...",Negativa
...,...,...,...,...
7775,7775,71327,감동적이기도하네요보고문물흘렸음,Positiva
7776,7776,27493,poor Malkovich,Positiva
7777,7777,133110,지자야닌의 전작 '초콜릿'과 비교하면 장난삼아 만든 영화인듯;; 별 두개도 후하다,Negativa
7778,7778,78429,시간가는줄 모르고 봤다..,Positiva


In [9]:
#creiamo le liste di train e test. 1.8k train 250 test  
train_texts=train2['Sentence']
train_labels=train2['Polarity_Classification']
test_texts=test['Sentence']
test_labels=test['Polarity_Classification']


<br><br>

## **Split the data into training and test sets**

In [10]:
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(67561, 67561, 7780, 7780)

Here's an example of a training label and review:

In [11]:
train_labels[0], train_texts[0]

('Negativa',
 "It's often choppy or unclear and I would imagine that its targetted age group would have difficulty following his train of thought")

<br><br>

## **Encode data for BERT**

We're going to transform our texts and labels into a format that BERT (via Huggingface and PyTorch) will understand. This is called *encoding* the data.

Here are the steps we need to follow:

1. The labels&mdash;in this case, Goodreads genres&mdash;need to be turned into integers rather than strings.

2. The texts&mdash;in this case, Goodreads reviews&mdash;need to be truncated if they're more than 512 tokens or padded if they're fewer than 512 tokens. The tokens, or words in the texts, also need to be separated into "word pieces" and matched to their embedding vectors.

3. We need to add special tokens to help BERT:

| BERT special token | Explanation |
| --------------| ---------|
| [CLS] | Start token of every document. |
| [SEP] | Separator between each sentence |
| [PAD] | Padding at the end of the document as many times as necessary, up to 512 tokens |
|  &#35;&#35; | Start of a "word piece" |




Here we will load `DistilBertTokenizerFast` from the HuggingFace library, which will do all the work of encoding the texts for us. The `tokenizer()` will break word tokens into word pieces, truncate to 512 tokens, and add padding and special BERT tokens.

In [12]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Here we will create a map of our labels, or Goodreads genres, to integer keys. We take the unique labels, and then we make a dictionary that associates each label/tag with an integer.

**Note:** HuggingFace documentation sometimes refers to "labels" as "tags" but these are the same thing. We use "labels" throughout this notebook for clarity.

In [17]:
unique_labels={'Positiva', 'Negativa'}
label2id={'Negativa': 0,  'Positiva': 1}
id2label={0:'Negativa', 1:'Positiva'}

In [18]:
unique_labels

{'Negativa', 'Positiva'}

In [19]:
label2id.keys()

dict_keys(['Negativa', 'Positiva'])

In [20]:
id2label.keys()

dict_keys([0, 1])

Now let's encode our texts and labels!

In [21]:
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
test_encodings  = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=max_length)

train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]

**Examine a Goodreads review in the training set after encoding**

In [22]:
' '.join(train_encodings[29].tokens[0:100])

'[CLS] It keeps you on the edge of your seat [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

**Examine a Goodreads review in the test set after encoding**

In [23]:
' '.join(test_encodings[0].tokens[0:100])

"[CLS] My dar ##ling , My ham ##burger is a good book if it didn ' t live you hang at the end what hope to whom and did they go to coll ##age or not [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

**Examine the training labels after encoding**

In [24]:
set(train_labels_encoded)

{0, 1}

**Examine the test labels after encoding**

In [25]:
set(test_labels_encoded)

{0, 1}

<br><br>

## **Make a custom Torch dataset**

Here we combine the encoded labels and texts into dataset objects. We use the custom Torch `MyDataSet` class to make a `train_dataset` object from  the `train_encodings` and `train_labels_encoded`. We also make a `test_dataset` object from `test_encodings`, and `test_labels_encoded`.

In [26]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [27]:
train_dataset = MyDataset(train_encodings, train_labels_encoded)
test_dataset = MyDataset(test_encodings, test_labels_encoded)

In [28]:
' '.join(train_dataset.encodings[0].tokens[0:100])

"[CLS] It ' s often cho ##ppy or unclear and I would ima ##gine that its target ##ted age group would have difficulty following his train of thought [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

In [29]:
' '.join(test_dataset.encodings[1].tokens[0:100])

'[CLS] I highly re ##com ##mend this book to any parent that does not know how to express the many things that they went through as a teen ##ager to their children [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

<br><br>

## **Load pre-trained BERT model**

Here we load a pre-trained DistilBERT model and send it to CUDA.

**Note:** If you decide to repeat fine-tuning after already running the following cells, make sure that you re-run this cell to re-load the original pre-trained model before fine-tuning again.

In [30]:
# The model_name needs to match the name used for the tokenizer above.
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.weight', 'classif

<br><br>

## **Set the BERT fine-tuning parameters**

These are the arguments we'll set in the HuggingFace TrainingArguments objects, which we'll then pass to the HuggingFace Trainer object. There are many more possible arguments, but here we highlight the basics and some common gotchas.

When training your own model, you should search over these parameters to find the best settings for your particular dataset. You should use a held-out set of validation data for this step.

| Parameter | Explanation |
|-----------| ------------|
| num_train_epochs | total number of training epochs (how many times to pass through the entire dataset; too much can cause overfitting) |
| per_device_train_batch_size | batch size per device during training |
| per_device_eval_batch_size |  batch size for evaluation |
|  warmup_steps |  number of warmup steps for learning rate scheduler (set lower because of small dataset size) |
| weight_decay | strength of weight decay (reduces size of weights, like regularization) |
| output_dir | output directory for the fine-tuned model and configuration files |
| logging_dir | directory for storing logs |
| logging_steps | how often to print logging output (so that we can stop training early if the loss isn't going down) |
| evaluation_strategy | evaluate while training so that we can see the accuracy going up |

In [31]:
training_args = TrainingArguments(
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=3e-5,              # initial learning rate for Adam optimizer
    warmup_steps=1000,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=5000,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)

<br><br>

## **Fine-tune the BERT model**

First, we define a custom evaluation function that returns the accuracy. You could modify this function to return precision, recall, F1, and/or other metrics.

In [32]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

Then we create a HuggingFace `Trainer` object using the `TrainingArguments` object that we created above. We also send our `compute_metrics` function to the `Trainer` object, along with our test and train datasets.

**Note:** This is what we've been aiming for this whole time! All the work of tokenizing, creating datasets, and setting the training arguments was for this cell.

In [33]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset (usually a validation set; here we just send our test set)
    compute_metrics=compute_metrics      # our custom evaluation function 
)

Time to finally fine-tune! 

Be patient; if you've set everything in Colab to use GPUs, then it should only take a minute or two to run, but if you're running on CPU, it can take hours.

After every 10 steps (as we specified in the TrainingArguments object), the trainer will output the current state of the model, including the training loss, validation ("test") loss, and accuracy (from our `compute_metrics` function).

You should see the loss going down and the accuracy going up. If instead they are staying the same or oscillating, you probably need to change the fine-tuning parameters.

In [34]:
trainer.train() #con 67k no neut        

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 67561
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 8446


Step,Training Loss,Validation Loss,Accuracy
5000,0.425300,0.270816,0.891645


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-3000
Configuration saved in ./results/checkpoint-3

TrainOutput(global_step=8446, training_loss=0.3701669276113508, metrics={'train_runtime': 7491.4646, 'train_samples_per_second': 18.037, 'train_steps_per_second': 1.127, 'total_flos': 1.0942320957606168e+16, 'train_loss': 0.3701669276113508, 'epoch': 2.0})

<br><br>

## **Save fine-tuned model**

The following cell will save the model and its configuration files to a directory in Colab. To preserve this model for future use, you should download the model to your computer.

In [35]:
trainer.save_model(cached_model_directory_name)

Saving model checkpoint to Polarity_2_noneut
Configuration saved in Polarity_2_noneut/config.json
Model weights saved in Polarity_2_noneut/pytorch_model.bin


(Optional) If you've already fine-tuned and saved the model, you can reload it using the following line. You don't have to run fine-tuning every time you want to evaluate.

<br><br>

## **Evaluate fine-tuned model**

The following function of the `Trainer` object will run the built-in evaluation, including our `compute_metrics` function.

In [36]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 7780
  Batch size = 16


{'epoch': 2.0,
 'eval_accuracy': 0.9093830334190232,
 'eval_loss': 0.2369605004787445,
 'eval_runtime': 147.097,
 'eval_samples_per_second': 52.89,
 'eval_steps_per_second': 3.311}

But we might want to do more fine-grained analysis of the model, so we extract the predicted labels.

In [37]:
predicted_results = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 7780
  Batch size = 16


In [38]:
predicted_results.predictions.shape

(7780, 2)

In [39]:
predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

In [40]:
len(predicted_labels)

7780

In [41]:
print(classification_report(test_labels, 
                            predicted_labels))

              precision    recall  f1-score   support

    Negativa       0.91      0.90      0.90      3663
    Positiva       0.91      0.92      0.91      4117

    accuracy                           0.91      7780
   macro avg       0.91      0.91      0.91      7780
weighted avg       0.91      0.91      0.91      7780



In [42]:
def prediction_test(dataset,modello):  #dataset fa riferimento al set di dati che vogliamo passare, modello dipende dal tipo di FT
#poi ci sarà anche la funzione per predire senza labels
    tok = DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased')
    test_texts=dataset['Sentence']
    test_labels=dataset['Polarity_Classification']

#unique_labels = set(label for label in test_labels)
#label2id = {label: id for id, label in enumerate(unique_labels)}
#id2label = {id: label for label, id in label2id.items()}
    unique_labels={'Positiva', 'Negativa'}
    label2id={'Negativa':0, 'Positiva': 1}
    id2label={0:'Negativa', 1:'Positiva'}

#train_encodings = tok(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
    test_encodings  = tok(test_texts.tolist(), truncation=True, padding=True, max_length=max_length)

#train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
    test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]
    

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

#train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)
    
    trainer = Trainer(model=modello)  #basta avere il modello come parametro

    predicted_results=trainer.predict(test_dataset)
    
    predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
    predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
    predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

#len(predicted_labels)

    return print(classification_report(test_labels,predicted_labels))

In [43]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
#tok = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.
tok = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.
mod = DistilBertForSequenceClassification.from_pretrained(cached_model_directory_name)

loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/28e5b750bf4f39cc620367720e105de1501cf36ec4ca7029eba82c1d2cc47caf.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/5cbdf121f196be5f1016cb102b197b0c34009e1e658f513515f2eebef9f38093.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/47087d99feeb3bc6184d7576ff089c52f7fbe

In [44]:
prediction_test(test[:335],mod)   #tabella per inglesi

loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/28e5b750bf4f39cc620367720e105de1501cf36ec4ca7029eba82c1d2cc47caf.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/5cbdf121f196be5f1016cb102b197b0c34009e1e658f513515f2eebef9f38093.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/47087d99feeb3bc6184d7576ff089c52f7fbe

              precision    recall  f1-score   support

    Negativa       0.94      0.79      0.86        98
    Positiva       0.92      0.98      0.95       237

    accuracy                           0.92       335
   macro avg       0.93      0.88      0.90       335
weighted avg       0.92      0.92      0.92       335



In [45]:
#per le inglesi e quindi domain book, l'accuracy e le altre statistiche per neutre e positive sono ottime
#per le negative le statistiche sono sufficientemente buone, anche per il fatto che il campione è piccolo

#risultati migliori rispetto a strategia 1 di training (soloEN)

In [47]:
prediction_test(test[335:3780],mod)   #tabella per italiane, domain social media e hotel reviews

loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/28e5b750bf4f39cc620367720e105de1501cf36ec4ca7029eba82c1d2cc47caf.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/5cbdf121f196be5f1016cb102b197b0c34009e1e658f513515f2eebef9f38093.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/47087d99feeb3bc6184d7576ff089c52f7fbe

              precision    recall  f1-score   support

    Negativa       0.95      0.95      0.95      1551
    Positiva       0.96      0.96      0.96      1894

    accuracy                           0.95      3445
   macro avg       0.95      0.95      0.95      3445
weighted avg       0.95      0.95      0.95      3445



In [ ]:
#risultati top per italia. Le neutre più che sufficientemente buone

In [48]:
prediction_test(test[3780:],mod)   #tabella per koreane, domain movie reviews

loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/28e5b750bf4f39cc620367720e105de1501cf36ec4ca7029eba82c1d2cc47caf.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/5cbdf121f196be5f1016cb102b197b0c34009e1e658f513515f2eebef9f38093.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/47087d99feeb3bc6184d7576ff089c52f7fbe

              precision    recall  f1-score   support

    Negativa       0.88      0.86      0.87      2014
    Positiva       0.86      0.88      0.87      1986

    accuracy                           0.87      4000
   macro avg       0.87      0.87      0.87      4000
weighted avg       0.87      0.87      0.87      4000



In [ ]:
#il modello è ottimo anche per coreano